<a href="https://colab.research.google.com/github/ko74dev/argo/blob/main/sktime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Классификация агрокультур 
на основе изменения показателя вегетационных индексов во временной последовательности

# Знакомимся с датасетом

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('train_dataset_train.csv', index_col='id')
df[-2:]

,area,nd_mean_2021-04-16,nd_mean_2021-04-19,nd_mean_2021-04-22,nd_mean_2021-04-26,nd_mean_2021-04-28,nd_mean_2021-05-02,nd_mean_2021-05-04,nd_mean_2021-05-07,nd_mean_2021-05-16,...,nd_mean_2021-05-20,nd_mean_2021-06-04,nd_mean_2021-06-05,nd_mean_2021-06-10,nd_mean_2021-07-05,nd_mean_2021-08-13,nd_mean_2021-08-27,nd_mean_2021-05-08,nd_mean_2021-05-24,crop
id,,,,,,,,,,,,,,,,,,,,,
5390,14,0.291399,0.221428,0.007247,0.055074,0.057781,0.187749,0.112403,0.183555,0.504538,...,0.536825,0.644357,0.085861,0.631304,0.694524,0.031836,0.453151,0.0,0.006495,5
860,75,0.226925,0.043076,0.058611,0.440539,0.120169,0.138846,0.020173,0.117669,0.266987,...,0.085106,0.003272,0.000000,0.010312,0.227649,0.000000,0.096585,0.0,0.011459,6


##  Выделяем исходные признаки и целевую переменную

In [ ]:
Y = df['crop']

# переименуем столбцы и отбросим area, geo
X = df.drop(columns=['area', '.geo', 'crop'])
X = X.rename(columns={c: pd.to_datetime('-'.join(c.split('_')[2:])) for c in X.columns})
X.sort_index(axis=1, inplace=True)

X.describe()

,2021-04-15,2021-04-16,2021-04-18,2021-04-19,2021-04-20,2021-04-22,2021-04-23,2021-04-25,2021-04-26,2021-04-27,...,2021-07-29,2021-07-31,2021-08-01,2021-08-07,2021-08-10,2021-08-11,2021-08-12,2021-08-13,2021-08-23,2021-08-27
count,4830.000000,4830.000000,4830.000000,4830.000000,4830.000000,4830.000000,4830.000000,4830.000000,4830.000000,4830.000000,...,4830.000000,4830.000000,4830.000000,4830.000000,4830.000000,4830.000000,4830.000000,4830.000000,4830.000000,4830.000000
mean,0.141455,0.149911,0.144619,0.118843,0.117383,0.158939,0.088569,0.102169,0.166146,0.096091,...,0.403969,0.322940,0.368242,0.261678,0.135701,0.204823,0.454516,0.121483,0.167788,0.145211
std,0.107284,0.112598,0.117913,0.103192,0.107475,0.136001,0.084578,0.100976,0.123605,0.099096,...,0.273735,0.279857,0.284816,0.255297,0.172781,0.223281,0.221768,0.151392,0.193051,0.162711
min,-0.013998,0.000000,-0.024201,0.000000,0.000000,0.000000,-0.029975,-0.012024,0.000000,-0.045503,...,0.000000,-0.007003,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.036935,0.047477,0.037164,0.025280,0.021316,0.050124,0.011723,0.017425,0.053859,0.013766,...,0.112063,0.058198,0.079748,0.039089,0.018811,0.027813,0.354782,0.015292,0.025991,0.022416
50%,0.135840,0.142240,0.120562,0.097346,0.088839,0.129819,0.070405,0.077609,0.155571,0.071599,...,0.473087,0.226646,0.386196,0.144791,0.080259,0.115949,0.509578,0.076428,0.099738,0.088429
75%,0.238828,0.245596,0.253963,0.202955,0.192879,0.241323,0.143344,0.157719,0.269131,0.146339,...,0.643702,0.591087,0.641436,0.523022,0.167927,0.358803,0.615458,0.162134,0.217370,0.198258
max,0.573308,0.987671,0.829457,0.760245,0.550173,0.764437,0.622755,0.805738,0.983779,0.810929,...,0.940949,0.938893,0.942515,0.850070,0.814770,0.871752,0.915747,0.896011,0.915447,0.786905


 ##  Переводим в нужный формат

In [ ]:
def to_format(data):
    cols = ["instances", "timepoints", 'var_0']
    items = []
    num_item = 0
    for row in data.itertuples():
        item=[]
        for i in range(1, len(row)):
            item.append([num_item, i, row[i]])

        num_item+=1
        items.append(pd.DataFrame(item, columns=cols))
    return pd.concat(items).set_index(["instances", "timepoints"])

In [ ]:
X = to_format(X)

## Тренируем модели классификаторов библиотеки sktime
### WEASEL

In [ ]:
from sktime.classification.dictionary_based import WEASEL

W = WEASEL(anova=False, bigrams=True, binning_strategy='information-gain', 
           window_inc=4, p_threshold=1, alphabet_size=4, n_jobs=1, 
           feature_selection='chi2', support_probabilities=False, random_state=42
    )
W.fit(X, Y)

/usr/local/lib/python3.7/dist-packages/sktime/classification/dictionary_based/_weasel.py:205: UserWarning: ``alphabet_size=4`` was deprecated in version 0.13.3 and will be changed to ``alphabet_size=2`` in 0.15.Please use alphabet_size=2 due to its lower memory footprint, better runtime at equal accuracy.
  "``alphabet_size=4`` was deprecated in version 0.13.3 and "
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,


WEASEL(anova=False, p_threshold=1, random_state=42, window_inc=4)

#### Предсказание результата

In [ ]:
test = pd.read_csv('test_dataset_test.csv', index_col='id')
test = to_format(test.drop(columns=['area','.geo']))

In [ ]:
pd.Series(W.predict(test)).to_csv('W.csv')

### MUSE

In [ ]:
from sktime.classification.dictionary_based import MUSE

M = MUSE(anova=False, variance=True, bigrams=False, 
         window_inc=4, alphabet_size=4, use_first_order_differences=False, 
         feature_selection='chi2', p_threshold=1, support_probabilities=False, 
         n_jobs=1, random_state=42 )
M.fit(X, Y)
pd.Series(M.predict(test)).to_csv('M.csv')

/usr/local/lib/python3.7/dist-packages/sktime/classification/dictionary_based/_muse.py:202: UserWarning: MUSE Warning: Input series is univariate; MUSE is designed for multivariate series. It is recommended WEASEL is used instead.
  "MUSE Warning: Input series is univariate; MUSE is designed for"
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,


### IndividualTDE

In [ ]:
from sktime.classification.dictionary_based import IndividualTDE

IDTE = IndividualTDE()
IDTE.fit(X, Y)

pd.Series(IDTE.predict(test)).to_csv('IDTE.csv')

### TemporalDictionaryEnsemble

In [ ]:
from sktime.classification.dictionary_based import TemporalDictionaryEnsemble

TDE = TemporalDictionaryEnsemble(n_parameter_samples=10,
                                  max_ensemble_size=3,
                               randomly_selected_params=5,)
TDE.fit(X, Y)

pd.Series(TDE.predict(test)).to_csv('TDE.csv')

### SupervisedTimeSeriesForest

In [ ]:
from sktime.classification.interval_based import SupervisedTimeSeriesForest

STSF = SupervisedTimeSeriesForest()
STSF.fit(X, Y)

pd.Series(STSF.predict(test)).to_csv('STSF.csv')

### RandomIntervalSpectralEnsemble

In [ ]:
from sktime.classification.interval_based import RandomIntervalSpectralEnsemble

RISE = RandomIntervalSpectralEnsemble()
RISE.fit(X, Y)

pd.Series(RISE.predict(test)).to_csv('RISE.csv')

### ShapeletTransformClassifier

In [ ]:
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime.classification.sklearn import RotationForest

STC = ShapeletTransformClassifier(estimator=RotationForest(n_estimators=3),
    n_shapelet_samples=100,
    max_shapelets=10,
    batch_size=20,)
STC.fit(X, Y)

pd.Series(STC.predict(test)).to_csv('STC.csv')

### Arsenal

In [ ]:
from sktime.classification.kernel_based import Arsenal

ars = Arsenal()
ars.fit(X, Y)

pd.Series(ars.predict(test)).to_csv('ars0.csv')

## Cамый популярный прогноз
Далее, по результатам прогнозов моделей, находился самый популярный прогноз.

In [ ]:
# Читаем прогнозы моделей (были перенесены в папку sktime)
csvs = !ls sktime

In [ ]:
subs = pd.read_csv('sample_solution.csv', index_col='id')
subs.drop(['crop'], axis=1, inplace=True)

In [ ]:
for f in csvs[:-1]:
    c = f.split('.')[0]
    subs[c] = 0
    subs[c].iloc[:] = pd.read_csv(f'sktime/{f}').iloc[:,-1].values

### Итоговый прогноз

In [ ]:
sub = pd.read_csv('sample_solution.csv', index_col='id')
sub['crop'] = subs.mode(1)[0].astype(int)
sub.to_csv('skt.csv', index=True)